# How to do Backpropagation with numpy

In [ ]:
#!pip install matplotlib
import matplotlib.pyplot as plt
import numpy as np


## Layer class

Like they provide a *forward* function, all our different layer types will get an *backward* function as well. Here we pass the upstream gradient as well as the original inputs to the function. Based on this the Layer will calculate the downstream gradient. 

In [ ]:
class Layer:
    """
    An identity Layer to be the base of inheritance
    """
    def __init__(self):
        pass
    
    def forward(self, input):
        # identity, same output as input
        return input
    
    def backward(self, input, grad_output, verbose=False):
        # The gradient of a dummy layer is precisely grad_output, but we'll write it more explicitly
        grad_output

## Updating the Sigmoid Function 

The derivative of the function

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\sigma(x) = \frac{1}{1+e^{-x}}$ 

is 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\sigma'(x) = \sigma(x) \cdot (1-\sigma(x))$

The chain rule says:
$\frac{\partial L}{\partial input} = \frac{\partial L}{\partial output} \cdot \frac{\partial output}{\partial input} = upstream gradient * local gradient$ 

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x)*(1- sigmoid(x))

class Sigmoid(Layer):
    def __init__(self):
        pass
    
    def forward(self, input):
        return sigmoid(input)
    
    def backward(self, input, grad_output, verbose=False):
        # apply the cain rule
        return grad_output * derivative_sigmoid(input) 
    
    def __str__(self):
        return "Sigmoid()"

Out input can be a vector or a matrix. Usually it will be a matrix of $batchsize \times inputnodes$. Thus the gradients will be of size $batchsize \times outputnodes$.

# Implmenting a RELU Layer

ReLU is similar to $max$ regarding backpropagation. 

In [ ]:
class ReLU(Layer):
    def __init__(self):
        # ReLU layer simply applies elementwise rectified linear unit to all inputs
        pass
    
    def forward(self, input):
        # Apply elementwise ReLU to [batch, input_units] matrix
        return np.maximum(0,input)
    
    def backward(self, input, grad_output):
        # Compute gradient of loss w.r.t. ReLU input
        relu_grad = input > 0
        return grad_output*relu_grad

# Backward function for the Dense Layer

In the activation functions we just backpropagated gradients, we had no weights to update. This time we do both 
* first we calculate gradients with respect to inputs (weights fixed)
* then we calculate the gradient with respect to weights (inputs fixed)
* we calculate the gradient with respect to the bias values (inputs fixed)
* Finally weights and biases are updated consigering gradients and learning rate

In [ ]:
class Dense(Layer):
    def __init__(self, input_units, output_units, learning_rate):
        # initialize weights with small random numbers. We use normal initialization, 
        self.weights = np.random.randn(input_units, output_units)*0.01
        self.biases = np.zeros(output_units)
        self.learning_rate=learning_rate
                    
    def forward(self,input):
        return input.dot(self.weights)+self.biases;
    
    def backward(self,input,grad_output,verbose=False):
        #compute downstream gradient = gradient with respect to inputs
        grad_input = np.dot(grad_output, self.weights.T) 
        
        #compute gradient with respect to weights and biases
        grad_weights = np.dot(input.T,grad_output)/input.shape[0] 
        grad_biases = grad_output.mean(axis=0) 
        
        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape
        
        #now we can sounchronuously update the weights = one stochastic gradient descent step. 
        self.weights = self.weights - self.learning_rate*grad_weights
        self.biases = self.biases - self.learning_rate*grad_biases
        
        return grad_input
    
    def __str__(self):
        text = f"Dense Layer {self.weights.shape}"
        return text        

## Integrating a gradient descent step to the network

Our first network shall just be able to learn the "and"-Function for binary values (0,1). So it can be rather small with just one layer so its easy to debug. Later we will increase the number of layers.

* During forward we will store all the inputs/outputs between the layers
* A dedicated predict function returns the outputs of the last layer
* We have to implement the derivative of the loss function
* We implement "trainBatch" which does one gradient descent step 

In [ ]:
class Network:
    def __init__(self, learning_rate): 
        self.layers = []
        self.layers.append(Dense(2,1, learning_rate))
        self.layers.append(Sigmoid())
        print("Number of layers", len(self.layers))
        
    def __str__(self):
        text = "Network:\n"
        for lay in self.layers:
            text = text + "  " +lay.__str__() + "\n"
        return text
    
    def forward(self, X):
        activations = [X]
        for layer in self.layers:
            X = layer.forward(X);
            activations.append(X)
        assert len(activations) == len(self.layers)+1
        #print("activations.shape", activations.shape)
        return activations

    def predict(self, input, verbose=False):
        return self.forward(input)[-1]
 
    def loss(self, y, pred):
        assert y.shape == pred.shape
        vec = -np.log(pred)*y-(np.log(1-pred)*(1-y))
        return np.mean(vec);
    
    # https://stats.stackexchange.com/questions/428228/how-does-cross-entropy-log-loss-work-with-backpropagation
    def log_loss_prime(self, y_true, y_pred):
        return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)
    
    def trainBatch(self, X, y, verbose = False):
        layer_inputs = self.forward(X)
        pred = layer_inputs[-1];
        if verbose: 
            print("layer_inputs")
            for lay in layer_inputs:
                print(lay)
            print("predictions", pred)

        # Compute the loss and the initial gradient
        loss_grad=self.log_loss_prime(y,pred)
        if verbose: print("Shape ", loss_grad.shape, " Value:", loss_grad)
    
        index = -2;
        for layer in reversed(self.layers):
            loss_grad = layer.backward(layer_inputs[index],loss_grad, verbose=verbose) #grad w.r.t. input, also weight updates
            if (verbose):
                print("Layer:", index, layer);
                print("Loss_grad:", loss_grad);

            index = index -1
        
        return self.loss(y, pred);

## Make a prediction with an untrained model 

In [ ]:
net = Network(0.1)
input = np.array([
    [1.0,1.0],
    [1.0,0.0],
    [0.0,1.0],
    [0.0,0.0],
]) 
y = np.array([1.0,0.0,0.0,0.0]).reshape(4,1)
pred = net.predict(input)
print("prediction",pred)
assert y.shape == pred.shape
print("loss",net.loss(y,pred))

print(net)

## Do gradient descent with multiple iterations

In [ ]:
net.trainBatch(input,y)
#print(net)
for i in range(10):
    for j in range(10000):
        net.trainBatch(input,y)
    print(net.trainBatch(input,y))
    
#print(net)



## Predict again with trained weights

In [ ]:
pred = net.predict(input)
print("prediction",pred) 
assert y.shape == pred.shape
print("loss",net.loss(y,pred))

# Exercises

* Implement a "fit" method to the Network class for training
* Experiment with the learning-rate and iterations
* Show that this model is still a linear model and plot the decision border
* Train a model that does the AND-Function on floatvalues like in the forward path